# Loading Ticker Data

Alpaca comes with a Polygon access api. Polygon is a provider of premium ticker data for most international tradeable assets. We are interested in accessing minute ticker data for each of these assets going back a few years. We will arbitrarily choose 2013-12-01 as our lookback date, primarily because we want enough training data without overfitting to historic trends. We can safely hypothesize that most recent ticker data will be most relevant for our model.

In [1]:
# Load packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import datetime
import seaborn as sns
np.random.seed(0)

api = tradeapi.REST(
    base_url=os.environ['APCA_API_BASE_URL'],
    key_id=os.environ['APCA_API_KEY_ID'],
    secret_key=os.environ['APCA_API_SECRET_KEY']
)

In [2]:
# Pick ETF Universe
symbols = [
    'XLF', # Financials
    'GDX', # Gold miners
    'VXX', # Volatility (Options)
    'EEM', # Emerging Markets
    'XRT', # S&P Retail
    'VTI', # Vanguard Total Stock Market
    'EWJ', # Japanese Market 
    'FXI', # China Large Cap
    'XHB', # S&P Homebuilders (Tracks real estate)
    'TLT', # 20 yr Treasury Bond
    'USO', # US Oil Fund
    'DBC', # Commodity Tracking
    'GLD', # Gold
    'SPY', # S&P 500
    'QQQ', # Nasdaq 100
    'XSW', # Computer Software
]

# Choose date range
lookback, lookforward = pd.to_datetime('2013-12-01 00:00:00'), pd.to_datetime('2019-06-07 00:00:00')

def get_min_ticker_data(lookback=lookback, lookforward=lookforward, symbols=symbols):
    """ Gets minute ticker data between two dates for list of ticker symbols. """
    hist = {}
    for symbol in symbols:
        point = lookback
        hist[symbol] = api.polygon.historic_agg(
                            size="minute", 
                            symbol=symbol, 
                            _from=str(point),
                            limit=5000,
                        ).df
        while point < lookforward:
            # Gets each tickers minute data
            point = hist[symbol].index[-1].tz_localize(None)
            hist[symbol] = pd.concat(
                [
                    hist[symbol],
                    api.polygon.historic_agg(
                            size="minute", 
                            symbol=symbol, 
                            _from=str(point),
                            limit=5000
                        ).df
                ],
                 axis=0)
        # Remove duplicates and sort index
        hist[symbol] = hist[symbol][~hist[symbol].index.duplicated(keep='last')].sort_index()
        # Easier OHLCV readability
        hist[symbol].columns = [c + '_' + symbol for c in hist[symbol].columns]
    return hist

This will take a second - we are requesting a large amount of data. Note that since we are requesting minute ticker data, certain times will have unavailable data, especially for the smaller ETFs in our list. To combat this, we will forward fill unavailable time frames, assuming that price has not changed, although it may have. 

In [3]:
hist = get_min_ticker_data()

In [5]:
df = pd.concat(hist.values(), axis=1).fillna(method='ffill')
df.dropna(inplace=True)
df.head()

open_XLF  high_XLF  low_XLF  close_XLF  volume_XLF  \
timestamp                                                                       
2013-12-02 09:30:00-05:00    21.510     21.53    21.50     21.520    104372.0   
2013-12-02 09:31:00-05:00    21.520     21.53    21.51     21.520     15900.0   
2013-12-02 09:32:00-05:00    21.510     21.52    21.49     21.495    753186.0   
2013-12-02 09:33:00-05:00    21.495     21.51    21.49     21.510     46024.0   
2013-12-02 09:34:00-05:00    21.510     21.52    21.51     21.520     47806.0   

                           open_GDX  high_GDX  low_GDX  close_GDX  volume_GDX  \
timestamp                                                                       
2013-12-02 09:30:00-05:00   21.8000   21.8400   21.780      21.83    257282.0   
2013-12-02 09:31:00-05:00   21.8400   21.8799   21.800      21.81    171214.0   
2013-12-02 09:32:00-05:00   21.8100   21.8100   21.770      21.78    286029.0   
2013-12-02 09:33:00-05:00   21.7799   21.8000   21.771      21.78    157994.0   
2013-12-02 09:34:00-05:00   21.7800   21.8000   21.770      21.79    125530.0   

                           ...  open_QQQ  high_QQQ  low_QQQ  close_QQQ  \
timestamp                  ...                                           
2013-12-02 09:30:00-05:00  ...     85.83    85.860  85.8100      85.85   
2013-12-02 09:31:00-05:00  ...     85.84    85.850  85.7900      85.80   
2013-12-02 09:32:00-05:00  ...     85.80    85.830  85.7800      85.81   
2013-12-02 09:33:00-05:00  ...     85.80    85.850  85.7985      85.85   
2013-12-02 09:34:00-05:00  ...     85.84    85.865  85.8300      85.85   

                           volume_QQQ  open_XSW  high_XSW  low_XSW  close_XSW  \
timestamp                                                                       
2013-12-02 09:30:00-05:00    271251.0    44.275    44.275   44.275     44.275   
2013-12-02 09:31:00-05:00    130647.0    44.275    44.275   44.275     44.275   
2013-12-02 09:32:00-05:00    185792.0    44.275    44.275   44.275     44.275   
2013-12-02 09:33:00-05:00    108899.0    44.275    44.275   44.275     44.275   
2013-12-02 09:34:00-05:00     92022.0    44.275    44.275   44.275     44.275   

                           volume_XSW  
timestamp                              
2013-12-02 09:30:00-05:00       360.0  
2013-12-02 09:31:00-05:00       360.0  
2013-12-02 09:32:00-05:00       360.0  
2013-12-02 09:33:00-05:00       360.0  
2013-12-02 09:34:00-05:00       360.0  

[5 rows x 80 columns]

In [6]:
df.shape

(1201242, 80)

In [7]:
df.to_csv('ticker_data.csv')